In [2]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader("faq.csv")

docs = loader.load()

In [4]:
docs[0]

Document(metadata={'source': 'faq.csv', 'row': 0}, page_content="Question: How can I reset my password?\nAnswer: Go to the login page and click on 'Forgot Password'. Enter your registered email address and follow the instructions sent to your inbox.")

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os 
from dotenv import load_dotenv
load_dotenv()
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-pro",api_key=os.environ["GOOGLE_API_KEY"],temperature=0.5)
response = llm.invoke("say my name")
print(response)

content="I would love to, but I don't know your name. I'm an AI and I don't have access to your personal information.\n\nIf you tell me your name, I'll be happy to say it" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--fad8af79-ff81-4621-ac8c-aa8f1e30b24e-0' usage_metadata={'input_tokens': 4, 'output_tokens': 47, 'total_tokens': 1028, 'input_token_details': {'cache_read': 0}}


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()


In [15]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(docs,embeddings)

vectordb.save_local("faq_faiss_index")

In [39]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS

vector_store = FAISS.load_local("faq_faiss_index", embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type ="similarity",search_kwargs = {"k":2})

memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",  
    return_messages=True       
)

prompt_template = """
You are a helpful FAQ assistant. Answer questions ONLY based on the provided FAQ information.
- Use the retrieved FAQ documents to answer the user's question.
- If the question is NOT covered in the FAQs, politely reply: 'I'm sorry, I don't know the answer to that question.'
- Be concise, clear, and professional in your answer.
- Maintain context using the conversation history if needed.

Conversation History:
{chat_history}

Retrieved FAQs:
{context}

User Question:
{question}

Answer:
"""

qa_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["chat_history","context","question"]
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriever,
    memory = memory,
    combine_docs_chain_kwargs = {"prompt":qa_prompt}
)

result_1 = qa_chain.invoke({"question": "Could you please assist me? I seem to have forgotten my password."})
print("Answer 1:", result_1['answer'])

result_2 = qa_chain.invoke({"question": "And how can I change my email?"})
print("Answer 2:", result_2['answer'])

result_3 = qa_chain.invoke({"question": "Who won the cricket world cup?"})  # Out-of-scope
print("Answer 3:", result_3['answer'])


Answer 1: To reset your password, go to the login page and click on 'Forgot Password'. Enter your registered email address and follow the instructions sent to your inbox.


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 40
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 37
}
]